# Step 2: Filling data gaps with IMDB API, data merge and cleaning

In [1]:
import numpy as np
import pandas as pd
from imdb import IMDb
from tqdm import tqdm

In [4]:
main = pd.read_csv('data/MoviesOnStreamingPlatforms.csv')
oscar = pd.read_csv('data/the_oscar_award.csv')
emmy = pd.read_csv('data/the_emmy_awards.csv')
# netflix = pd.read_csv('netflix_titles.csv')
# amazon = pd.read_csv('amazon_prime_titles.csv')

In [5]:
# amazon_type = amazon[['type', 'title']].set_index('title').type.to_dict()
# netflix_type = netflix[['type', 'title']].set_index('title').type.to_dict()
# types = amazon_type.copy()
# types.update(netflix_type)
# types

In [6]:
# amazon.type.value_counts()

In [7]:
# netflix.type.value_counts()

In [8]:
# main.Title.map(types).value_counts()

In [9]:
# Getting type values from the netflix and amazon datasets seems not very good
# due to few number of TV shows identified.

In [10]:
main.head()

,Unnamed: 0,ID,Title,Year,Age,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type
0,0,1,The Irishman,2019,18+,98/100,1,0,0,0,0
1,1,2,Dangal,2016,7+,97/100,1,0,0,0,0
2,2,3,David Attenborough: A Life on Our Planet,2020,7+,95/100,1,0,0,0,0
3,3,4,Lagaan: Once Upon a Time in India,2001,7+,94/100,1,0,0,0,0
4,4,5,Roma,2018,18+,94/100,1,0,0,0,0


In [11]:
main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9515 entries, 0 to 9514
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       9515 non-null   int64 
 1   ID               9515 non-null   int64 
 2   Title            9515 non-null   object
 3   Year             9515 non-null   int64 
 4   Age              5338 non-null   object
 5   Rotten Tomatoes  9508 non-null   object
 6   Netflix          9515 non-null   int64 
 7   Hulu             9515 non-null   int64 
 8   Prime Video      9515 non-null   int64 
 9   Disney+          9515 non-null   int64 
 10  Type             9515 non-null   int64 
dtypes: int64(8), object(3)
memory usage: 817.8+ KB


In [12]:
main.Type.unique()

array([0], dtype=int64)

In [13]:
# Select the columns required for the analyses & change the name of the Prime Video column to Amazon
cols = ['Title', 'Year', 'Age', 'Rotten Tomatoes', 'Netflix', 'Prime Video']
main_2 = main.loc[:,cols].rename(columns={'Prime Video': 'Amazon'})
main_2

,Title,Year,Age,Rotten Tomatoes,Netflix,Amazon
0,The Irishman,2019,18+,98/100,1,0
1,Dangal,2016,7+,97/100,1,0
2,David Attenborough: A Life on Our Planet,2020,7+,95/100,1,0
3,Lagaan: Once Upon a Time in India,2001,7+,94/100,1,0
4,Roma,2018,18+,94/100,1,0
...,...,...,...,...,...,...
9510,Most Wanted Sharks,2020,NaN,14/100,0,0
9511,Doc McStuffins: The Doc Is In,2020,NaN,13/100,0,0
9512,Ultimate Viking Sword,2019,NaN,13/100,0,0
9513,Hunt for the Abominable Snowman,2011,NaN,10/100,0,0


In [14]:
# Drop titles that are not on Netflix or Amazon
main_3 = main_2[(main_2['Netflix']==1) | (main_2['Amazon']==1)]
main_3

,Title,Year,Age,Rotten Tomatoes,Netflix,Amazon
0,The Irishman,2019,18+,98/100,1,0
1,Dangal,2016,7+,97/100,1,0
2,David Attenborough: A Life on Our Planet,2020,7+,95/100,1,0
3,Lagaan: Once Upon a Time in India,2001,7+,94/100,1,0
4,Roma,2018,18+,94/100,1,0
...,...,...,...,...,...,...
8610,Arthur and the Haunted Tree House,2017,NaN,13/100,0,1
8611,Rollin: The Fall of the Auto Industry and the ...,2016,NaN,13/100,0,1
8612,The Fairies: Fairy Beach,2007,all,12/100,0,1
8613,5K1-States Evidence,2018,NaN,12/100,0,1


In [15]:
# Emmy nominees that are found in the main dataset
emmy.loc[emmy['nominee'].isin(main_3.Title)].head()

,id,year,category,nominee,staff,company,producer,win
12,13,2019,Outstanding Production Design For A Narrative ...,Barry,"Tyler B. Robinson, Production Designer; Eric S...",HBO,HBO Entertainment in association with Alec Ber...,False
21,22,2019,Outstanding Casting For A Comedy Series,Barry,"Sherry Thomas, CSA, Casting by; Sharon Bialy, ...",HBO,HBO Entertainment in association with Alec Ber...,False
64,65,2019,Outstanding Directing For A Comedy Series,Barry,"Alec Berg, Directed by",HBO,HBO Entertainment in association with Alec Ber...,False
65,66,2019,Outstanding Directing For A Comedy Series,Barry,"Bill Hader, Directed by",HBO,HBO Entertainment in association with Alec Ber...,False
96,97,2019,Outstanding Single-Camera Picture Editing For ...,Barry,"Kyle Reiter, ACE, Editor",HBO,HBO Entertainment in association with Alec Ber...,False


In [16]:
emmy_1 = emmy.loc[emmy['nominee'].isin(main_3.Title) & emmy['year'].isin(main_3.Year)][['nominee', 'win']]
emmy_1.head()

,nominee,win
12,Barry,False
21,Barry,False
64,Barry,False
65,Barry,False
96,Barry,False


In [17]:
emmy_no_nomination = emmy_1.nominee.value_counts().to_dict()
main_3['emmy_nomination'] = main_3.Title.map(emmy_no_nomination).fillna(0).astype(int)

emmy_no_win = emmy_1.groupby('nominee').sum().win.to_dict()
main_3['emmy_win'] = main_3.Title.map(emmy_no_win).fillna(0).astype(int)

main_3

C:\Users\kimh\AppData\Local\Temp/ipykernel_19980/3406706024.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_3['emmy_nomination'] = main_3.Title.map(emmy_no_nomination).fillna(0).astype(int)
C:\Users\kimh\AppData\Local\Temp/ipykernel_19980/3406706024.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_3['emmy_win'] = main_3.Title.map(emmy_no_win).fillna(0).astype(int)


,Title,Year,Age,Rotten Tomatoes,Netflix,Amazon,emmy_nomination,emmy_win
0,The Irishman,2019,18+,98/100,1,0,0,0
1,Dangal,2016,7+,97/100,1,0,0,0
2,David Attenborough: A Life on Our Planet,2020,7+,95/100,1,0,0,0
3,Lagaan: Once Upon a Time in India,2001,7+,94/100,1,0,0,0
4,Roma,2018,18+,94/100,1,0,0,0
...,...,...,...,...,...,...,...,...
8610,Arthur and the Haunted Tree House,2017,NaN,13/100,0,1,0,0
8611,Rollin: The Fall of the Auto Industry and the ...,2016,NaN,13/100,0,1,0,0
8612,The Fairies: Fairy Beach,2007,all,12/100,0,1,0,0
8613,5K1-States Evidence,2018,NaN,12/100,0,1,0,0


In [18]:
oscar.head()

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False


In [19]:
oscar_1 = oscar.loc[oscar.film.isin(main_3.Title) & \
                    (oscar.year_film.isin(main_3.Year) | oscar.year_ceremony.isin(main_3.Year))][['film', 'winner']]
oscar_1.head()

,film,winner
25,The Jazz Singer,False
29,Underworld,True
39,The Patriot,False
46,The Patriot,False
61,The Patriot,False


In [20]:
oscar_no_nomination = oscar_1.film.value_counts().to_dict()
main_3['oscar_nomination'] = main_3.Title.map(oscar_no_nomination).fillna(0).astype(int)

oscar_no_win = oscar_1.groupby('film').sum().winner.to_dict()
main_3['oscar_win'] = main_3.Title.map(oscar_no_win).fillna(0).astype(int)

main_3

C:\Users\kimh\AppData\Local\Temp/ipykernel_19980/2633757729.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_3['oscar_nomination'] = main_3.Title.map(oscar_no_nomination).fillna(0).astype(int)
C:\Users\kimh\AppData\Local\Temp/ipykernel_19980/2633757729.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_3['oscar_win'] = main_3.Title.map(oscar_no_win).fillna(0).astype(int)


,Title,Year,Age,Rotten Tomatoes,Netflix,Amazon,emmy_nomination,emmy_win,oscar_nomination,oscar_win
0,The Irishman,2019,18+,98/100,1,0,0,0,10,0
1,Dangal,2016,7+,97/100,1,0,0,0,0,0
2,David Attenborough: A Life on Our Planet,2020,7+,95/100,1,0,0,0,0,0
3,Lagaan: Once Upon a Time in India,2001,7+,94/100,1,0,0,0,0,0
4,Roma,2018,18+,94/100,1,0,0,0,10,3
...,...,...,...,...,...,...,...,...,...,...
8610,Arthur and the Haunted Tree House,2017,NaN,13/100,0,1,0,0,0,0
8611,Rollin: The Fall of the Auto Industry and the ...,2016,NaN,13/100,0,1,0,0,0,0
8612,The Fairies: Fairy Beach,2007,all,12/100,0,1,0,0,0,0
8613,5K1-States Evidence,2018,NaN,12/100,0,1,0,0,0,0


In [21]:
# # https://www.imdb.com/interfaces/
# imdb = pd.read_csv('title.basics.tsv', sep='\t')
# imdb.head()

In [22]:
# imdb.titleType.unique()

In [23]:
# # primaryTitle (string) – the more popular title / the title used by the filmmakers on promotional materials at the point of release
# # originalTitle (string) - original title, in the original language
# imdb_1 = imdb.replace('\\N', np.NaN).drop(['originalTitle', 'isAdult', 'runtimeMinutes'], axis=1)
# # endYear (YYYY) – TV Series end year. ‘\N’ for all other title types
# imdb_1[imdb_1['endYear'].notnull()].titleType.unique()

In [24]:
# # Simply take 'movie' and 'tvMovie' as 'movie' and 'tvSeries'&'tvMiniSeries' as 'tv'
# imdb_2 = imdb_1[(imdb_1['titleType'].isin(['movie', 'tvMovie', 'tvSeries', 'tvMiniSeries'])) & \
#                 (imdb_1['startYear'].astype(float)>=main_3.Year.min())]
# imdb_2['titleType'].replace({'tvMovie':'movie', 'tvSeries':'tv', 'tvMiniSeries':'tv'}, inplace=True)
# imdb_2.head()

In [25]:
# imdb_ratings = pd.read_csv('title.ratings.tsv', sep='\t')
# imdb_ratings.head()

In [26]:
# imdb_dict = imdb_ratings.set_index('tconst').averageRating.to_dict()
# imdb_2['IMDb'] = imdb_2.tconst.map(imdb_dict)
# imdb_2.rename(columns={'titleType':'Type', 'primaryTitle':'Title', 'startYear':'Year', 'genres':'Genres'}, inplace=True)
# imdb_2['Year']=imdb_2.Year.astype(int)
# imdb_2.head()

In [27]:
# merged = pd.merge(main_3, imdb_2[['Type', 'Title', 'Year', 'Genres', 'IMDb']], on=['Title', 'Year'], how='left')
# merged

In [28]:
# merged.info()

In [29]:
# merged.describe()

In [30]:
# merged_2 = pd.merge(merged, df[['Title', 'Year', 'IMDb', 'Age', 'Genres']], on=['Title', 'Year'], how='left')
# merged_2

In [31]:
# # Using originalTitle instead of primaryTitle results in a less complete merge.

# imdb_9 = imdb.replace('\\N', np.NaN)[imdb['endYear'].notnull()]

# imdb_10 = imdb_9[(imdb_9['titleType'].isin(['movie', 'tvMovie', 'tvSeries', 'tvMiniSeries'])) & \
#                 (imdb_9['startYear'].astype(float)>=main_3.Year.min())]
# imdb_10['titleType'].replace({'tvMovie':'movie', 'tvSeries':'tv', 'tvMiniSeries':'tv'}, inplace=True)

# imdb_10['IMDb'] = imdb_10.tconst.map(imdb_dict)
# imdb_10.rename(columns={'titleType':'Type', 'originalTitle':'Title', 'startYear':'Year', 'genres':'Genres'}, inplace=True)
# imdb_10['Year']=imdb_10.Year.astype(int)

# merged_1 = pd.merge(main_3, imdb_10[['Type', 'Title', 'Year', 'Genres', 'IMDb']], on=['Title', 'Year'], how='left')
# merged_1.info()

In [32]:
# try this for getting the IMDb data directly:
# https://www.kaggle.com/ruchi798/movies-data-collection-eda-using-tableau/notebook
# https://imdbpy.readthedocs.io/en/latest/usage/movie.html

In [41]:
imdb = IMDb()
kind_list = []
rating_list = []
genres_list = []
total = len(main_3)
for title, year in tqdm(zip(main_3.Title, main_3.Year)):
    title_year = title + ' (' + str(year) + ')'
    movie_search = imdb.search_movie(title_year)
    if len(movie_search)!=0:
        movie_get = imdb.get_movie(movie_search[0].getID())
        
        try:
            kind = movie_get['kind']
        except:
            kind = np.NaN
            
        try:
            rating = movie_get['rating']
        except:
            rating = np.NaN
            
        try:
            genres = movie_get['genres']
        except:
            genres = np.NaN
            
        kind_list.append(kind)
        rating_list.append(rating)
        genres_list.append(genres)
    
    else:
        kind_list.append(np.NaN)
        rating_list.append(np.NaN)
        genres_list.append(np.NaN)

7699it [11:34:21,  5.41s/it]


In [42]:
main_3['Type'] = kind_list
main_3['IMDb'] = rating_list
main_3['Genres'] = genres_list

C:\Users\kimh\AppData\Local\Temp/ipykernel_19980/3188220225.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_3['Type'] = kind_list
C:\Users\kimh\AppData\Local\Temp/ipykernel_19980/3188220225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  main_3['IMDb'] = rating_list
C:\Users\kimh\AppData\Local\Temp/ipykernel_19980/3188220225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats i

In [ ]:
# After running the above cells I had problems with saving the dataframe as CSV
# hence I had to save the data in an unconventional way like below.
# Since the API call above takes too long to complete and
# there were some modifications to this Notebook after 'main.csv' was saved,
# so the 'main.csv' that will be created by this Notebook might be a little different,
# but as long as one uses the most recent Notebook (Streaming_video_analysis_MASTER-Copy1.ipynb) and 
# the csv file (main_final_2.csv), there shouldn't be a problem.
# Please consider this as an experimental, intermediate script/file.

# file = open('type.txt', 'w')
# for i in kind_list:
#     file.write(str(i) + '\n')
# file.close()

In [ ]:
# file = open('rating.txt', 'w')
# for i in rating_list:
#     file.write(str(i) + '\n')
# file.close()

In [ ]:
# file = open('genre.txt', 'w')
# for i in genres_list:
#     file.write(str(i) + '\n')
# file.close()

In [ ]:
# kind_list = []
# with open('type.txt') as f:
#     lines = f.readlines()
#     for line in lines:
#         kind_list.append(line[:-1])
# # kind_list

In [ ]:
# rating_list = []
# with open('rating.txt') as f:
#     lines = f.readlines()
#     for line in lines:
#         rating_list.append(line[:-1])
# # rating_list

In [ ]:
# genre_list = []
# with open('genre.txt') as f:
#     lines = f.readlines()
#     for line in lines:
#         genre_list.append(line[0:-1].strip("][").replace("'", "").split(', '))
# # genre_list

In [ ]:
# main_3.Type.replace({"nan": np.NaN}, inplace=True)
# main_3.IMDb.replace({"nan": np.NaN}, inplace=True)
# main_3.Genres.replace({"nan": np.NaN}, inplace=True)
# main_3.info()

In [43]:
main_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7699 entries, 0 to 8614
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Title             7699 non-null   object 
 1   Year              7699 non-null   int64  
 2   Age               3997 non-null   object 
 3   Rotten Tomatoes   7692 non-null   object 
 4   Netflix           7699 non-null   int64  
 5   Amazon            7699 non-null   int64  
 6   emmy_nomination   7699 non-null   int32  
 7   emmy_win          7699 non-null   int32  
 8   oscar_nomination  7699 non-null   int32  
 9   oscar_win         7699 non-null   int32  
 10  Type              7490 non-null   object 
 11  IMDb              7204 non-null   float64
 12  Genres            7473 non-null   object 
dtypes: float64(1), int32(4), int64(3), object(5)
memory usage: 721.8+ KB


In [44]:
main_3.head()

,Title,Year,Age,Rotten Tomatoes,Netflix,Amazon,emmy_nomination,emmy_win,oscar_nomination,oscar_win,Type,IMDb,Genres
0,The Irishman,2019,18+,98/100,1,0,0,0,10,0,movie,7.8,"[Biography, Crime, Drama]"
1,Dangal,2016,7+,97/100,1,0,0,0,0,0,movie,8.4,"[Action, Biography, Drama, Sport]"
2,David Attenborough: A Life on Our Planet,2020,7+,95/100,1,0,0,0,0,0,movie,9.0,"[Documentary, Biography]"
3,Lagaan: Once Upon a Time in India,2001,7+,94/100,1,0,0,0,0,0,movie,8.1,"[Drama, Musical, Sport]"
4,Roma,2018,18+,94/100,1,0,0,0,10,3,movie,7.7,[Drama]


In [56]:
pd.read_csv('(original) main.csv').drop('Unnamed: 0', axis=1)

,Title,Year,Age,Rotten Tomatoes,Netflix,Amazon,emmy_nomination,emmy_win,oscar_nomination,oscar_win,Type,IMDb,Genres
0,The Irishman,2019,18+,98/100,1,0,0,0,10,0,movie,7.8,"['Biography', 'Crime', 'Drama']"
1,Dangal,2016,7+,97/100,1,0,0,0,0,0,movie,8.4,"['Action', 'Biography', 'Drama', 'Sport']"
2,David Attenborough: A Life on Our Planet,2020,7+,95/100,1,0,0,0,0,0,movie,9.0,"['Documentary', 'Biography']"
3,Lagaan: Once Upon a Time in India,2001,7+,94/100,1,0,0,0,0,0,movie,8.1,"['Drama', 'Musical', 'Sport']"
4,Roma,2018,18+,94/100,1,0,0,0,10,3,movie,7.7,['Drama']
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7694,Arthur and the Haunted Tree House,2017,NaN,13/100,0,1,0,0,0,0,tv movie,5.8,['Animation']
7695,Rollin: The Fall of the Auto Industry and the ...,2016,NaN,13/100,0,1,0,0,0,0,movie,6.5,"['Documentary', 'Crime', 'News']"
7696,The Fairies: Fairy Beach,2007,all,12/100,0,1,0,0,0,0,tv movie,NaN,['Family']
7697,5K1-States Evidence,2018,NaN,12/100,0,1,0,0,0,0,NaN,NaN,['nan']
